# Importing necessary modules

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import math
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
%matplotlib inline

# Reading the dataset

In [2]:
df=pd.read_csv('Data_Science_Internship - Dump.csv', index_col=0)

In [3]:
df.head()

,Agent_id,status,lost_reason,budget,lease,movein,source,source_city,source_country,utm_source,utm_medium,des_city,des_country,room_type,lead_id
0,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Not responding,NaN,NaN,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,ecc0e7dc084f141b29479058967d0bc07dee25d9690a98...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,cd5dc0d9393f3980d11d4ba6f88f8110c2b7a7f7796307...
1,299ae77a4ef350ae0dd37d6bba1c002d03444fb1edb236...,LOST,Low budget,NaN,NaN,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,5372372f3bf5896820cb2819300c3e681820d82c6efc54...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,b94693673a5f7178d1b114e4004ad52377d3244dd24a3d...
2,c213697430c006013012dd2aca82dd9732aa0a1a6bca13...,LOST,Not responding,£121 - £180 Per Week,Full Year Course Stay 40 - 44 weeks,31/08/22,7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2...,9b8cc3c63cdf447e463c11544924bf027945cbd29675f7...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,11ab03a1a8c367191355c152f39fe28cae5e426fce49ef...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,Ensuite,96ea4e2bf04496c044745938c0299c264c3f4ba079e572...
3,eac9815a500f908736d303e23aa227f0957177b0e6756b...,LOST,Low budget,0-0,0,NaN,ba2d0a29556ac20f86f45e4543c0825428cba33fd7a9ea...,a5f0d2d08eb0592087e3a3a2f9c1ba2c67cc30f2efd2bd...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,19372fa44c57a01c37a5a8418779ca3d99b0b59731fb35...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,1d2b34d8add02a182a4129023766ca4585a8ddced0e5b3...
4,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Junk lead,NaN,NaN,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,NaN,fc10fffd29cfbe93c55158fb47752a7501c211d253468c...


Notes: '9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0' represents NaN 

In [4]:
df=df.replace('9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0', np.nan)

Note: The leads with STATUS other than ‘WON’ or ‘LOST’ can be dropped during training.

In [5]:
df=df[(df.status =='LOST')|( df.status=='WON')]

# Manually Label Encoding Target Variable

In [6]:
df['status']=list(map(lambda x: 1 if x=='WON' else 0, df['status']))

# Dropping duplicates

In [7]:
df=df.drop_duplicates()

# Inspect columns

In [8]:
df.columns

Index(['Agent_id', 'status', 'lost_reason', 'budget', 'lease', 'movein',
       'source', 'source_city', 'source_country', 'utm_source', 'utm_medium',
       'des_city', 'des_country', 'room_type', 'lead_id'],
      dtype='object')

'Agent_id': Unique Identifier of Agent responsible for dealing with the client, 

'status': Present status of the lead, 

'lost_reason': Reason why they lost the lead, 

'budget': Budget of the lead, 

'lease': Lease duration lead is willing to engage in (assuming the numbers without any text represent weeks), 

'movein': Move in date ,

'source', 'source_city', 'source_country', : Source address of the lead

'utm_source', 'utm_medium',
       
 'des_city', 'des_country': address where the lead is about to move in, 
 
 'room_type': type of room lead is interested in, 
 
 'lead_id': unique identifier of a lead

# Plan of action

we are going to create a logistic regression model that can classify our records into WON or LOST. and we can use it to give us a probability which we can use for Lead Score

# Hypothesis
1. Some agents might have better success rate than others
2. budget might be a strong reason for a lead to final the deal or not
3. lease and des_city might be a influential factor in deciding to forward with a deal or not
4. room type may also be a reason when combined with budget

# As the dataset is imbalanced, Separating dataset into majority and minority

# columns to consider
1. Agent_id
2. Budget
3. Lease
4. source_city
5. utm_medium
6. des_city
7. des_country
5. room_type

In [9]:
imp_cols=['Agent_id', 'budget', 'lease', 'source', 'source_city', 'utm_medium',
       'des_city', 'des_country', 'room_type']

# Candidates with status as WON has  NULL in lost_reason it is a calssic case of causation and nor correlation. So even though this column is highly related we will discard it in training our model

# budget column 

# Assigning slab to budget column

In [10]:
def assign_budget(budget):
    nums=[]
    
    if budget=='unavailable':
        return('unavailable')
    temp=re.findall('\d*',budget)
    
    
    for item in temp:
        if item!='':
            nums.append(int(item))
            
    if len(nums)==0:
        return '0-60'
           
    high=max(nums)
    
    if high<=60:
        return '0-60'
    elif high>60 and high<=120:
        return '61-120'
    elif high>120 and high<=180:
        return '121-180'
    elif high>180 and high<=240:
        return '181-240'
    elif high>240 and high<=300:
        return '240-300'
    elif high>300:
        return '>300'
    else:
        return 'other'
        

In [11]:
budget_mode=df.budget.mode()[0]


# lease

# Assigning slab to lease column

In [12]:
def find_week_slab(weeks):
    if weeks>=52:
        return('1 year or more')
    elif weeks>=39 and weeks<52:
        return('9 months - 1 year')
    elif weeks>=26 and weeks<39:
        return('6 months - 9 months')
    elif weeks>=13 and weeks<26:
        return('3 months - 6 months')
    elif weeks<13:
        return('less than 3 months')
    else:
        return('Other')

In [13]:
def categorize_lease_term(lease):
    if lease=='unavailable':
        return 'unavailable'
    nums=[]
    #print(lease)
    if re.search('year',lease)!=None or re.search('course',lease)!=None :
        return('1 year or more')
        
    elif re.search('month',lease)!=None:
        temp=re.findall('\d*',lease)
        for item in temp:
            if item!='':
                nums.append(int(item))
            
            return(find_week_slab(round(max(nums)*4.3)))
    
    else:
        
        temp=re.findall('\d*',lease)
    
        for item in temp:
            if item!='':
                nums.append(int(item))

        if len(nums)==0:
            return 'Other'

        high=max(nums)
        return(find_week_slab(max(nums)))
        

# DES_CITY

# Keeping cities with more than 100 records and putting others in other_des_cities category

In [14]:
des_cities=df.des_city.value_counts()[df.des_city.value_counts().values>100].index
des_cities

Index(['ecc0e7dc084f141b29479058967d0bc07dee25d9690a98ee4e6fdad5168274d7',
       '810c069040f6a9b16fdf976a901755508a87cb0224b169d4b5d803357e647947',
       '11ab03a1a8c367191355c152f39fe28cae5e426fce49efb320230ca4ae3f97a1',
       '8593b9db65212160979d946950601c26622a219c80f1f122679eb69ec5b96600',
       '6a5ed83658ef85afc77709cc16ede854ad98e3c2ad8b076e10dcbcd6e5096271',
       '5e02dadaa7e4cd29809e2f8a115f1e5e9c01b29813a1fb9707218804218da32a',
       '9b8cc3c63cdf447e463c11544924bf027945cbd29675f77955bb36364356c14e',
       'b2586a6cef5690b74e9fb425f95f8fb3f1e18a4cdc3225eb2f53534ec3602aee',
       '5372372f3bf5896820cb2819300c3e681820d82c6efc548fb07bed3a6c83f451',
       'c384826dfbfce62c3fd99a46984a56fe71546a85c2cb6cf12031691b8e038a23',
       '085c3a49a31cbfa71c36746a98dc908d30d423a63b969c5a5c7cadefcd92492b',
       'bf5ade33b5b472c3599473a5cb4e5c7037e4c73296b7e97006edb71f029133b6',
       '9edff2628ff7385bffbe7fc9cae89010d2cb9182e805e146de99e99cd4dfce61',
       '1b4e390dcd79bc059

# Source city

# Keeping cities with more than 100 records and putting others in other_source_cities category

In [15]:
source_cities=df['source_city'].value_counts()[df['source_city'].value_counts().values>100].index
source_cities

Index(['ecc0e7dc084f141b29479058967d0bc07dee25d9690a98ee4e6fdad5168274d7',
       'b384863fa1f6e091164b74219726eee0b9dd47776b91a4dc71fd0516630d21ec',
       '9f2ba6981e36ea0fca6c16f42e6413d788d2a7499b8b7f418c5d76f12d191f02',
       '7d1550b844ff586a6023216c06263105eed0a849a2a1f69bb8862ab288d8cdab',
       '810c069040f6a9b16fdf976a901755508a87cb0224b169d4b5d803357e647947',
       '4fa64bd55d5c0c1f83015952b4b9500cb099dd0b1b04647f1a7c3c5708295b6e',
       '282f96b099630502f8bb5033849c69982dd147015853061ed938c2a254fd8dcf',
       '5e02dadaa7e4cd29809e2f8a115f1e5e9c01b29813a1fb9707218804218da32a',
       '8593b9db65212160979d946950601c26622a219c80f1f122679eb69ec5b96600',
       'b2586a6cef5690b74e9fb425f95f8fb3f1e18a4cdc3225eb2f53534ec3602aee',
       '7b7f15f9ffbeb905caaee265ac39d267d0512ab5333f2fa25d523204af13c5a6',
       '9b8cc3c63cdf447e463c11544924bf027945cbd29675f77955bb36364356c14e',
       'c384826dfbfce62c3fd99a46984a56fe71546a85c2cb6cf12031691b8e038a23',
       '11ab03a1a8c367191

# Source

In [16]:
sources=df['source'].value_counts()[df['source'].value_counts().values>100].index
sources

Index(['7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2b7c51be879f0c37ca8',
       '7bcfca0e9d73371699b0adbf1c691e02108fa64f02f4dbe24a0217f139a9b916',
       'ba2d0a29556ac20f86f45e4543c0825428cba33fd7a9eaa29e3f169d2ca43b2b',
       '146fb4ecbe78caa24102bbaac12e2559a8c8e32fb116d36e3553a709fce40549',
       'd684761c17c11590f6e2525b48141cb2c0c6f2be5df4e229dae06e64c5c41b64',
       '9ba9134a91cfc6b52ac8d480e9ad37896ca4ac216e2d795fdb7d75a63d6c60af',
       '9fd09dc33545f9cc19b81ebd0b98c4fd8c66ed1e34de89f4c9a81e6b26dc0d54',
       'b2b2a0ecb072ed25f1844a3325a810b85689bcc785ddb40dbdcd50a237e40831',
       'ece6174b48090f929b5c750eb652162e2f0861a90b843446eb61318030aea9b3',
       '869105a4ce02fc56897c8d7e1eb37736b95e3e826568ae3172eabaf582f69707',
       '76b7fd432636403ce93adce5cef7871bc24d064c374493aa098686d0be3df5d3',
       'b5d2d728817d666ce97f6944eae23cff0bae5f81b1917b2e3056d73d9f9fd601',
       'fce2de0ca752918e030db0b31521693134ecfbf1c32e1402715169aa4bf16086',
       'dfe8ecc0693503f62

# utm_source

# Marking null utm_source as 'unavailable'

# function to process dataframe

In [17]:
def process_df(df):    
    df=df.replace('9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0', np.nan)
    
    df['budget'].fillna(budget_mode, inplace=True)
    df['budget']=df['budget'].apply(assign_budget)
    
    df['utm_medium'].fillna('unavailable', inplace=True)
    
    df['lease'].fillna('unavailable', inplace=True)
    df['lease']=df['lease'].apply(categorize_lease_term)
    
    df['des_city'].fillna('unavailable', inplace=True)
    df['des_city']=list(map(lambda x: 'Other des cities' if x not in des_cities else x, df['des_city']))
    
    df['des_country'].fillna('unavailable', inplace=True)
    
    df['source_city'].fillna('unavailable', inplace=True)
    df['source_city']=list(map(lambda x: 'Other Source Cities' if x not in source_cities else x, df['source_city']))
    
    df['source']=list(map(lambda x: 'Other Sources' if x not in sources else x, df['source']))
    df.utm_medium.fillna('unavailable', inplace=True)
    
    df.room_type.fillna('not_specified', inplace=True)
    return df

In [18]:
df.shape

(46307, 15)

In [19]:
X=df[imp_cols]
X=process_df(X)
Y=df['status']

# Checking independent variables for consistency before training data

In [20]:
X.isna().sum()

Agent_id       0
budget         0
lease          0
source         0
source_city    0
utm_medium     0
des_city       0
des_country    0
room_type      0
dtype: int64

In [21]:
X.nunique()

Agent_id       120
budget           6
lease            7
source          15
source_city     56
utm_medium      64
des_city        51
des_country     15
room_type        6
dtype: int64

In [22]:
X=pd.get_dummies(X)


In [23]:
Y.value_counts(1)

0    0.93366
1    0.06634
Name: status, dtype: float64

In [24]:
train_X,test_X,train_y,test_y= train_test_split(X,Y, test_size=0.1, stratify=Y, random_state=22)

In [25]:
model_lr=LogisticRegression(max_iter=1000)

In [26]:
model_lr.fit(train_X,train_y)

LogisticRegression(max_iter=1000)

In [27]:
y_pred=model_lr.predict(test_X)


In [28]:
print(classification_report(test_y,y_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4324
           1       0.59      0.03      0.06       307

    accuracy                           0.93      4631
   macro avg       0.76      0.52      0.51      4631
weighted avg       0.91      0.93      0.91      4631



In [29]:
accuracy_score(test_y,y_pred),precision_score(test_y,y_pred),recall_score(test_y,y_pred),f1_score(test_y,y_pred)

(0.9343554307924854,
 0.5882352941176471,
 0.03257328990228013,
 0.06172839506172839)

# Our model is performing badly because of the imbalanced dataset. Upsampling using SMOTE to balance the dataset

In [30]:
X=df[imp_cols]
X=process_df(X)
X=pd.get_dummies(X)
y=df['status']
from imblearn.over_sampling import SMOTE
oversample = SMOTE()

X, y = oversample.fit_resample(X, y)

In [31]:
X.shape

(86470, 340)

In [32]:
y.value_counts()

0    43235
1    43235
Name: status, dtype: int64

In [33]:
train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.1, random_state=22)
model_lr=LogisticRegression(solver='lbfgs', max_iter=1000)
model_lr.fit(train_X,train_y)
y_pred=model_lr.predict(test_X)
print(classification_report(test_y,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      4334
           1       0.99      0.92      0.96      4313

    accuracy                           0.96      8647
   macro avg       0.96      0.96      0.96      8647
weighted avg       0.96      0.96      0.96      8647



# Oversampling has resulted in maximising the performance of our model

# Now we can use the proba function to find out the probability of a client going through the lease and will use the same value as our score to mark our customers

In [34]:
df=pd.read_csv('Data_Science_Internship - Dump.csv', index_col=0)

In [35]:
encoded_df=process_df(df)

In [36]:
encoded_df.nunique()

Agent_id            120
status                6
lost_reason          30
budget                6
lease                 7
movein              477
source               15
source_city          56
source_country      185
utm_source           34
utm_medium           64
des_city             51
des_country          15
room_type             6
lead_id           30574
dtype: int64

In [37]:
encoded_df=encoded_df[imp_cols]

In [38]:
encoded_df.nunique()

Agent_id       120
budget           6
lease            7
source          15
source_city     56
utm_medium      64
des_city        51
des_country     15
room_type        6
dtype: int64

In [39]:
encoded_df=pd.get_dummies(encoded_df)

In [40]:
pred=model_lr.predict_proba(encoded_df)

In [41]:
pred.shape

(46608, 2)

In [42]:
pred

array([[0.99575274, 0.00424726],
       [0.95708656, 0.04291344],
       [0.89528097, 0.10471903],
       ...,
       [0.97173862, 0.02826138],
       [0.99625954, 0.00374046],
       [0.93685679, 0.06314321]])

In [43]:
df['score']=[round(pred[i][1]*100,2) for i in range(pred.shape[0])]

In [44]:
df.score

0         0.42
1         4.29
2        10.47
3        12.10
4         1.13
         ...  
46603     1.23
46604     2.19
46605     2.83
46606     0.37
46607     6.31
Name: score, Length: 46608, dtype: float64

In [45]:
print("Minimum score:{}, Maximum score:{}".format(min(df.score), max(df.score)))

Minimum score:0.0, Maximum score:90.23


In [46]:
df.to_excel('result.xlsx', index=0)